# Training Workflow

Here we show the process of processing a query request.It includes the following steps：
    1. Connect DO nodes and MPC nodes.
    2. Load model.
    3. Train model.
    4. query completed.

### Preparation

We need start the websocket server workers.

### Step 1: get info from response

由于我们需要展示一个处理查询的流程，因此我们首先需要创建一个包含查询信息的结构体。
首先导入依赖包。

In [4]:
import datetime
import sys

sys.path.append('../../../')
from soterml.connection.proto import soterone_data_pb2, soterone_service_pb2
from soterml.learning.horizontal.torch_model.testnet_models import Net1

ModuleNotFoundError: No module named 'soterone_data_pb2'

In [2]:
import socket
def get_host_ip():
    """
    query native IP
    :return: ip
    """
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()

    return ip

def get_query_info_and_nodes_info():
    cur_time = str(datetime.date.today())
    mock_response = soterone_service_pb2.GetQueryInfoResponse()
    query_info = mock_response.query_info
    query_info.uuid = 'uuid ' + cur_time
    query_info.content = ('content ' + cur_time).encode('utf8')

    query_cluster = mock_response.query_cluster
    query_cluster.query_uuid = 'query_cluster.query_uuid ' + cur_time

    # MPC nodes
    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8779"

    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8780"
    
    mpc_nodes = query_cluster.mpc_nodes.add()
    mpc_nodes.address = get_host_ip()
    mpc_nodes.port = "8781"

    # Compute nodes
    do_nodes = query_cluster.do_nodes.add()
    do_nodes.address = get_host_ip()
    do_nodes.port = "8777"

    do_nodes = query_cluster.do_nodes.add()
    do_nodes.address = get_host_ip()
    do_nodes.port = "8778"

    mock_response.query_info.data_set = 1

    return mock_response

In [3]:
query_info_and_nodes_info = get_query_info_and_nodes_info()
model = Net1()

NameError: name 'soterone_service_pb2' is not defined

In [4]:
import syft
import torch
hook = syft.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'd:\programdata\anaconda3\envs\python37\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [5]:
from soterml.learning.testnet.utils import get_nodes_from_query_info
compute_nodes, mpc_nodes = get_nodes_from_query_info(query_info_and_nodes_info, hook)

In [ ]:
from soterml.learning.testnet.MNIST_MPC_websocket_parallel import train_model
model_trained = train_model(chain,
                            query_uuid,
                            query_info_and_nodes_info,
                            compute_nodes,
                            model,
                            mpc_nodes)